In [75]:
import geopandas as gpd

import numpy as np
import pandas as pd
from pandas import IndexSlice as idx
import tensorflow as tf
import sys
import os
import glob

from functools import partial



code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/perturbations/'
sys.path.append(code_dir)
code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/diff_bpr'
sys.path.append(code_dir)
from top_k import top_k_idx
#from make_datasets import make_data
from bpr_model import PerturbedBPRModel


code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/'
sys.path.append(code_dir)
from zinf_gp.metrics import normcdf, fixed_top_X



from perturbations import perturbed
from bpr import bpr_variable_k_no_ties



In [2]:
data_path='/cluster/tufts/hugheslab/datasets/NSF_OD/results_20220606_update/clean_quarter_tract/'

In [113]:
def make_data_quarterly(multiindexed_gdf, first_year, last_year, time_window, feature_cols, train_shape, pred_lag=1):


    xs = []
    ys = []

    for eval_year in range(first_year, last_year + 1):
        quarters_in_year = multiindexed_gdf[multiindexed_gdf['year']==eval_year].index.unique(level='timestep')
        quarters_in_year.sort_values()
        train_x_df = multiindexed_gdf.loc[idx[:, min(quarters_in_year) - time_window:max(quarters_in_year) - pred_lag], feature_cols]

        

        

        for quarter in quarters_in_year:
            
            train_x_df['pred_timestep'] = quarter
            train_x_vals = train_x_df.values.reshape(train_shape)
            
            train_y_df = multiindexed_gdf.loc[idx[:,quarter], 'deaths']
            train_y_vals = train_y_df.values

            xs.append(train_x_vals)
            ys.append(train_y_vals)

    x_BSTD = np.stack(xs, axis=0)
    y_BS = np.stack(ys)

    x_BSTD = tf.convert_to_tensor(x_BSTD, dtype=tf.float32)
    y_BS = tf.convert_to_tensor(y_BS, dtype=tf.float32)

    B, S, T, D = x_BSTD.shape

    assert (B == len(range(first_year, last_year + 1))*pred_lag)
    assert (S == train_shape[0])
    assert (T == time_window)
    assert (D == len(feature_cols)+1)

    # Reshape the training data to flatten the dimensions
    x_BSF_flat = tf.reshape(x_BSTD, (B, S, T * D), )


    return x_BSF_flat, y_BS

In [115]:
class PerturbedBPRModel(tf.keras.Model):

    def __init__(self, perturbed_top_k_func, k=100):
        """k should match the k baked into the perturbed top_k func.
        we need k for when performing exact top k in evaluation step."""
        super(PerturbedBPRModel, self).__init__()
        self.perturbed_top_k_func = perturbed_top_k_func
        self.k = k
        self.hidden1 = tf.keras.layers.Dense(100, activation='relu')
        self.hidden2 = tf.keras.layers.Dense(50, activation='relu')
        self.hidden3 = tf.keras.layers.Dense(10, activation='relu')
        self.output_layer = tf.keras.layers.Dense(1, activation=None)

    def call(self, inputs):
        intermediate = self.hidden1(inputs)
        intermediate = self.hidden2(intermediate)
        intermediate = self.hidden3(intermediate)
        
        outputs = self.output_layer(intermediate)
        # squeeze away feature dimension
        outputs = tf.squeeze(outputs, axis=-1)
        return outputs

    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            top_100_indicators = self.perturbed_top_k_func(y_pred)
            true_top_100_val, true_top_100_idx = tf.math.top_k(y, k=self.k)

            denominator = tf.reduce_sum(true_top_100_val, axis=-1)
            numerator = tf.reduce_sum(top_100_indicators * y, axis=-1)

            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(numerator, denominator, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        # Unpack the data
        x, y = data
        # Compute predictions
        y_pred = self(x, training=False)  # Forward pass
        # use discrete topk to simulate making a decision
        _, pred_100_idx = tf.math.top_k(y_pred, k=self.k)
        true_top_100_val, true_top_100_idx = tf.math.top_k(y, k=self.k)

        denominator = tf.reduce_sum(true_top_100_val, axis=-1)
        numerator = tf.reduce_sum(tf.gather(y, pred_100_idx, batch_dims=-1), axis=-1)

        # Compute the loss value
        # (the loss function is configured in `compile()`)
        self.compiled_loss(numerator, denominator, regularization_losses=self.losses)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}

In [121]:
epochs = 5000
seed = 360
time_window = 5*4
first_train_eval_year = 2014
last_train_eval_year = 2018
#batch_dim_size = last_train_eval_year - first_train_eval_year + 1
first_validation_year = 2019
last_validation_year = 2019
first_test_year = 2020
last_test_year = 2021

tf.random.set_seed(seed)


timestep_col = 'timestep'
geography_col = 'geoid'
outcome_col = 'deaths'

x_idx_cols = [geography_col, 'lat', 'lon', timestep_col,
              'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
              'svi_pctile', 'year',
              'neighbor_t', 'deaths']
y_idx_cols = [geography_col, timestep_col, outcome_col]
features_only = ['lat', 'lon', timestep_col,
                 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
                 'svi_pctile',
                 'neighbor_t', 'deaths']
#features_only = ['deaths']

#data_gdf = gpd.read_file(data_path)

multiindexed_gdf = data_gdf.set_index(['geoid', 'timestep'])
multiindexed_gdf['timestep'] = multiindexed_gdf.index.get_level_values('timestep')
num_geoids = len(data_gdf['geoid'].unique())

train_shape = (num_geoids, time_window, len(features_only)+1)

train_x_BSF_flat, train_y_BS = make_data_quarterly(multiindexed_gdf, first_train_eval_year, last_train_eval_year,
                                                  time_window, features_only, train_shape, pred_lag=4)

valid_x_BSF_flat, valid_y_BS = make_data_quarterly(multiindexed_gdf, first_validation_year, last_validation_year,
                                         time_window, features_only, train_shape, pred_lag=4)

test_x_BSF_flat, test_y_BS = make_data_quarterly(multiindexed_gdf, first_test_year, last_test_year,
                                       time_window, features_only, train_shape, pred_lag=4)

norm_layer = tf.keras.layers.Normalization()
norm_layer.adapt(train_x_BSF_flat)
train_x_BSF_flat = norm_layer(train_x_BSF_flat)
valid_x_BSF_flat = norm_layer(valid_x_BSF_flat)
test_x_BSF_flat = norm_layer(test_x_BSF_flat)

top_100_idx_func = partial(top_k_idx, k=100)

In [144]:
perturbation_samples = 37
noise=0.4
learning_rate = 0.0001


In [145]:
perturbed_top_100 = perturbed(top_100_idx_func,
                              num_samples=perturbation_samples,
                              sigma=noise,
                              noise='normal',
                              batched=True)

model = PerturbedBPRModel(perturbed_top_100)

In [146]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Compile the model
def weird_loss(a, b):
    return -a / b

model.compile(optimizer=optimizer, loss=weird_loss)


In [ ]:

model.fit(train_x_BSF_flat, train_y_BS, epochs=epochs, batch_size=batch_dim_size,
          validation_data=(valid_x_BSF_flat, valid_y_BS))

Epoch 1/5000
1/1 [==============================] - 1s 977ms/step - loss: -0.2091 - val_loss: -0.3290
Epoch 2/5000
1/1 [==============================] - 0s 244ms/step - loss: -0.2085 - val_loss: -0.3330
Epoch 3/5000
1/1 [==============================] - 0s 243ms/step - loss: -0.2073 - val_loss: -0.3344
Epoch 4/5000
1/1 [==============================] - 0s 251ms/step - loss: -0.2100 - val_loss: -0.3345
Epoch 5/5000
1/1 [==============================] - 0s 246ms/step - loss: -0.2111 - val_loss: -0.3275
Epoch 6/5000
1/1 [==============================] - 0s 243ms/step - loss: -0.2099 - val_loss: -0.3261
Epoch 7/5000
1/1 [==============================] - 0s 249ms/step - loss: -0.2107 - val_loss: -0.3303
Epoch 8/5000
1/1 [==============================] - 0s 250ms/step - loss: -0.2144 - val_loss: -0.3370
Epoch 9/5000
1/1 [==============================] - 0s 252ms/step - loss: -0.2122 - val_loss: -0.3385
Epoch 10/5000
1/1 [==============================] - 0s 254ms/step - loss: -0.2148

1/1 [==============================] - 0s 250ms/step - loss: -0.2635 - val_loss: -0.3525
Epoch 81/5000
1/1 [==============================] - 0s 250ms/step - loss: -0.2623 - val_loss: -0.3498
Epoch 82/5000
1/1 [==============================] - 0s 255ms/step - loss: -0.2660 - val_loss: -0.3498
Epoch 83/5000
1/1 [==============================] - 0s 249ms/step - loss: -0.2642 - val_loss: -0.3454
Epoch 84/5000
1/1 [==============================] - 0s 251ms/step - loss: -0.2643 - val_loss: -0.3498
Epoch 85/5000
1/1 [==============================] - 0s 254ms/step - loss: -0.2644 - val_loss: -0.3485
Epoch 86/5000
1/1 [==============================] - 0s 249ms/step - loss: -0.2648 - val_loss: -0.3498
Epoch 87/5000
1/1 [==============================] - 0s 249ms/step - loss: -0.2651 - val_loss: -0.3498
Epoch 88/5000
1/1 [==============================] - 0s 252ms/step - loss: -0.2656 - val_loss: -0.3527
Epoch 89/5000
1/1 [==============================] - 0s 251ms/step - loss: -0.2655 - va

1/1 [==============================] - 0s 251ms/step - loss: -0.2856 - val_loss: -0.3486
Epoch 160/5000
1/1 [==============================] - 0s 253ms/step - loss: -0.2840 - val_loss: -0.3486
Epoch 161/5000
1/1 [==============================] - 0s 250ms/step - loss: -0.2850 - val_loss: -0.3444
Epoch 162/5000
1/1 [==============================] - 0s 247ms/step - loss: -0.2858 - val_loss: -0.3457
Epoch 163/5000
1/1 [==============================] - 0s 247ms/step - loss: -0.2836 - val_loss: -0.3457
Epoch 164/5000
1/1 [==============================] - 0s 250ms/step - loss: -0.2846 - val_loss: -0.3485
Epoch 165/5000
1/1 [==============================] - 0s 245ms/step - loss: -0.2846 - val_loss: -0.3471
Epoch 166/5000
1/1 [==============================] - 0s 242ms/step - loss: -0.2849 - val_loss: -0.3485
Epoch 167/5000
1/1 [==============================] - 0s 255ms/step - loss: -0.2826 - val_loss: -0.3459
Epoch 168/5000
1/1 [==============================] - 0s 250ms/step - loss: -0.

Epoch 238/5000
1/1 [==============================] - 0s 250ms/step - loss: -0.2959 - val_loss: -0.3452
Epoch 239/5000
1/1 [==============================] - 0s 252ms/step - loss: -0.2994 - val_loss: -0.3452
Epoch 240/5000
1/1 [==============================] - 0s 258ms/step - loss: -0.2974 - val_loss: -0.3452
Epoch 241/5000
1/1 [==============================] - 0s 255ms/step - loss: -0.2990 - val_loss: -0.3438
Epoch 242/5000
1/1 [==============================] - 0s 251ms/step - loss: -0.2988 - val_loss: -0.3451
Epoch 243/5000
1/1 [==============================] - 0s 253ms/step - loss: -0.2993 - val_loss: -0.3424
Epoch 244/5000
1/1 [==============================] - 0s 250ms/step - loss: -0.3001 - val_loss: -0.3424
Epoch 245/5000
1/1 [==============================] - 0s 247ms/step - loss: -0.3000 - val_loss: -0.3397
Epoch 246/5000
1/1 [==============================] - 0s 253ms/step - loss: -0.3003 - val_loss: -0.3397
Epoch 247/5000
1/1 [==============================] - 0s 250ms/s

Epoch 317/5000
1/1 [==============================] - 0s 251ms/step - loss: -0.3043 - val_loss: -0.3609
Epoch 318/5000
1/1 [==============================] - 0s 248ms/step - loss: -0.3025 - val_loss: -0.3625
Epoch 319/5000
1/1 [==============================] - 0s 248ms/step - loss: -0.3038 - val_loss: -0.3639
Epoch 320/5000
1/1 [==============================] - 0s 249ms/step - loss: -0.3033 - val_loss: -0.3626
Epoch 321/5000
1/1 [==============================] - 0s 250ms/step - loss: -0.3024 - val_loss: -0.3625
Epoch 322/5000
1/1 [==============================] - 0s 253ms/step - loss: -0.3052 - val_loss: -0.3625
Epoch 323/5000
1/1 [==============================] - 0s 246ms/step - loss: -0.3049 - val_loss: -0.3625
Epoch 324/5000
1/1 [==============================] - 0s 248ms/step - loss: -0.3053 - val_loss: -0.3568
Epoch 325/5000
1/1 [==============================] - 0s 248ms/step - loss: -0.3039 - val_loss: -0.3568
Epoch 326/5000
1/1 [==============================] - 0s 251ms/s

Epoch 396/5000
1/1 [==============================] - 0s 245ms/step - loss: -0.3022 - val_loss: -0.3609
Epoch 397/5000
1/1 [==============================] - 0s 245ms/step - loss: -0.3022 - val_loss: -0.3623
Epoch 398/5000
1/1 [==============================] - 0s 250ms/step - loss: -0.2998 - val_loss: -0.3596
Epoch 399/5000
1/1 [==============================] - 0s 255ms/step - loss: -0.3004 - val_loss: -0.3609
Epoch 400/5000
1/1 [==============================] - 0s 252ms/step - loss: -0.3007 - val_loss: -0.3638
Epoch 401/5000
1/1 [==============================] - 0s 255ms/step - loss: -0.3008 - val_loss: -0.3638
Epoch 402/5000
1/1 [==============================] - 0s 249ms/step - loss: -0.3018 - val_loss: -0.3611
Epoch 403/5000
1/1 [==============================] - 0s 245ms/step - loss: -0.3001 - val_loss: -0.3611
Epoch 404/5000
1/1 [==============================] - 0s 249ms/step - loss: -0.2988 - val_loss: -0.3596
Epoch 405/5000
1/1 [==============================] - 0s 253ms/s

Epoch 475/5000
1/1 [==============================] - 0s 248ms/step - loss: -0.2994 - val_loss: -0.3630
Epoch 476/5000
1/1 [==============================] - 0s 248ms/step - loss: -0.2976 - val_loss: -0.3643
Epoch 477/5000
1/1 [==============================] - 0s 246ms/step - loss: -0.2971 - val_loss: -0.3643
Epoch 478/5000
1/1 [==============================] - 0s 250ms/step - loss: -0.2982 - val_loss: -0.3643
Epoch 479/5000
1/1 [==============================] - 0s 251ms/step - loss: -0.2965 - val_loss: -0.3658
Epoch 480/5000
1/1 [==============================] - 0s 248ms/step - loss: -0.2979 - val_loss: -0.3658
Epoch 481/5000
1/1 [==============================] - 0s 254ms/step - loss: -0.2971 - val_loss: -0.3658
Epoch 482/5000
1/1 [==============================] - 0s 248ms/step - loss: -0.2969 - val_loss: -0.3616
Epoch 483/5000
1/1 [==============================] - 0s 250ms/step - loss: -0.2971 - val_loss: -0.3616
Epoch 484/5000
1/1 [==============================] - 0s 250ms/s

Epoch 554/5000
1/1 [==============================] - 0s 249ms/step - loss: -0.3035 - val_loss: -0.3625
Epoch 555/5000
1/1 [==============================] - 0s 244ms/step - loss: -0.3042 - val_loss: -0.3597
Epoch 556/5000
1/1 [==============================] - 0s 251ms/step - loss: -0.3036 - val_loss: -0.3610
Epoch 557/5000
1/1 [==============================] - 0s 251ms/step - loss: -0.3037 - val_loss: -0.3623
Epoch 558/5000
1/1 [==============================] - 0s 250ms/step - loss: -0.3046 - val_loss: -0.3637
Epoch 559/5000
1/1 [==============================] - 0s 256ms/step - loss: -0.3036 - val_loss: -0.3651
Epoch 560/5000
1/1 [==============================] - 0s 254ms/step - loss: -0.3068 - val_loss: -0.3651
Epoch 561/5000
1/1 [==============================] - 0s 252ms/step - loss: -0.3038 - val_loss: -0.3665
Epoch 562/5000
1/1 [==============================] - 0s 251ms/step - loss: -0.3031 - val_loss: -0.3665
Epoch 563/5000
1/1 [==============================] - 0s 246ms/s

Epoch 633/5000
1/1 [==============================] - 0s 250ms/step - loss: -0.3061 - val_loss: -0.3581
Epoch 634/5000
1/1 [==============================] - 0s 247ms/step - loss: -0.3051 - val_loss: -0.3581
Epoch 635/5000
1/1 [==============================] - 0s 255ms/step - loss: -0.3063 - val_loss: -0.3567
Epoch 636/5000
1/1 [==============================] - 0s 253ms/step - loss: -0.3054 - val_loss: -0.3540
Epoch 637/5000
1/1 [==============================] - 0s 252ms/step - loss: -0.3070 - val_loss: -0.3526
Epoch 638/5000
1/1 [==============================] - 0s 251ms/step - loss: -0.3069 - val_loss: -0.3513
Epoch 639/5000
1/1 [==============================] - 0s 252ms/step - loss: -0.3054 - val_loss: -0.3513
Epoch 640/5000
1/1 [==============================] - 0s 252ms/step - loss: -0.3067 - val_loss: -0.3526
Epoch 641/5000
1/1 [==============================] - 0s 251ms/step - loss: -0.3071 - val_loss: -0.3554
Epoch 642/5000
1/1 [==============================] - 0s 251ms/s

Epoch 712/5000
1/1 [==============================] - 0s 253ms/step - loss: -0.3054 - val_loss: -0.3574
Epoch 713/5000
1/1 [==============================] - 0s 247ms/step - loss: -0.3042 - val_loss: -0.3587
Epoch 714/5000
1/1 [==============================] - 0s 248ms/step - loss: -0.3059 - val_loss: -0.3587
Epoch 715/5000
1/1 [==============================] - 0s 255ms/step - loss: -0.3060 - val_loss: -0.3558
Epoch 716/5000
1/1 [==============================] - 0s 251ms/step - loss: -0.3061 - val_loss: -0.3573
Epoch 717/5000
1/1 [==============================] - 0s 253ms/step - loss: -0.3052 - val_loss: -0.3629
Epoch 718/5000
1/1 [==============================] - 0s 252ms/step - loss: -0.3057 - val_loss: -0.3642
Epoch 719/5000
1/1 [==============================] - 0s 248ms/step - loss: -0.3041 - val_loss: -0.3571
Epoch 720/5000
1/1 [==============================] - 0s 250ms/step - loss: -0.3058 - val_loss: -0.3543
Epoch 721/5000
1/1 [==============================] - 0s 253ms/s

In [139]:
model.trainable_variables

[<tf.Variable 'perturbed_bpr_model_11/dense_30/kernel:0' shape=(220, 100) dtype=float32, numpy=
 array([[ 0.00719592, -0.04645859, -0.01844875, ..., -0.04718753,
          0.03084159,  0.14573094],
        [-0.11836787, -0.12660244,  0.02698829, ..., -0.08388493,
         -0.14309686,  0.19262102],
        [-0.08026059, -0.28874218,  0.10318363, ..., -0.05022891,
          0.02977351,  0.3901074 ],
        ...,
        [ 0.23114634,  0.08450731, -0.03968769, ..., -0.20577903,
         -0.14238843, -0.15449598],
        [-0.13269791, -0.06289726,  0.2431028 , ...,  0.16756043,
         -0.20101394,  0.24970037],
        [-0.16971661, -0.25981182,  0.2715672 , ...,  0.07830626,
          0.02652317,  0.31739092]], dtype=float32)>,
 <tf.Variable 'perturbed_bpr_model_11/dense_30/bias:0' shape=(100,) dtype=float32, numpy=
 array([ 0.06196941,  0.161347  ,  0.26962522,  0.08456077, -0.12389129,
        -0.18943824, -0.16642849, -0.06824225, -0.01711683, -0.04803375,
        -0.12577438,  0.1

In [82]:
multiindexed_gdf

year  quarter  deaths  month STATEFP COUNTYFP TRACTCE   
geoid       timestep                                                          
25001010100 0.0       2000      1.0     0.0      1      25      001   10100  \
            1.0       2000      2.0     0.0      4      25      001   10100   
            2.0       2000      3.0     0.0      7      25      001   10100   
            3.0       2000      4.0     0.0     10      25      001   10100   
            4.0       2001      1.0     3.0      1      25      001   10100   
...                    ...      ...     ...    ...     ...      ...     ...   
25027761402 83.0      2020      4.0     0.0     10      25      027  761402   
            84.0      2021      1.0     1.0      1      25      027  761402   
            85.0      2021      2.0     0.0      4      25      027  761402   
            86.0      2021      3.0     0.0      7      25      027  761402   
            87.0      2021      4.0     0.0     10      25      027  761402   

                         NAME              NAMELSAD  MTFCC  ... TRACTCE_y   
geoid       timestep                                        ...             
25001010100 0.0           101      Census Tract 101  G5020  ...       NaN  \
            1.0           101      Census Tract 101  G5020  ...       NaN   
            2.0           101      Census Tract 101  G5020  ...       NaN   
            3.0           101      Census Tract 101  G5020  ...       NaN   
            4.0           101      Census Tract 101  G5020  ...       NaN   
...                       ...                   ...    ...  ...       ...   
25027761402 83.0      7614.02  Census Tract 7614.02  G5020  ...       NaN   
            84.0      7614.02  Census Tract 7614.02  G5020  ...       NaN   
            85.0      7614.02  Census Tract 7614.02  G5020  ...       NaN   
            86.0      7614.02  Census Tract 7614.02  G5020  ...       NaN   
            87.0      7614.02  Census Tract 7614.02  G5020  ...       NaN   

                      month_sinc   season  season_sin  qtr_since_  year_since   
geoid       timestep                                                            
25001010100 0.0              0.0  jan-jun         0.0         0.0         0.0  \
            1.0              3.0  jan-jun         0.0         1.0         0.0   
            2.0              6.0  jul-dec         1.0         2.0         0.0   
            3.0              9.0  jul-dec         1.0         3.0         0.0   
            4.0             12.0  jan-jun         2.0         4.0         1.0   
...                          ...      ...         ...         ...         ...   
25027761402 83.0           249.0  jul-dec        41.0        83.0        20.0   
            84.0           252.0  jan-jun        42.0        84.0        21.0   
            85.0           255.0  jan-jun        42.0        85.0        21.0   
            86.0           258.0  jul-dec        43.0        86.0        21.0   
            87.0           261.0  jul-dec        43.0        87.0        21.0   

                      self_t-1  neighbor_t   
geoid       timestep                         
25001010100 0.0            0.0       0.000  \
            1.0            0.0       0.000   
            2.0            0.0       0.000   
            3.0            0.0       0.000   
            4.0            0.0       0.000   
...                        ...         ...   
25027761402 83.0           0.0       0.125   
            84.0           0.0       0.125   
            85.0           0.0       0.000   
            86.0           0.0       0.000   
            87.0           0.0       0.375   

                                                               geometry   
geoid       timestep                                                      
25001010100 0.0       POLYGON ((-70.25001 42.06410, -70.24959 42.065...  \
            1.0       POLYGON ((-70.25001 42.06410, -70.24959 42.065...   
            2.0       POLYGON ((-70.250

In [122]:
train_x_BSF_flat

<tf.Tensor: shape=(20, 1620, 220), dtype=float32, numpy=
array([[[-0.7479397 ,  1.8678857 , -1.4142135 , ..., -1.0031025 ,
         -0.45583406, -1.647509  ],
        [-1.2349257 ,  2.1685455 , -1.4142135 , ..., -1.0031025 ,
         -0.45583406, -1.647509  ],
        [-0.9121866 ,  2.0892835 , -1.4142135 , ..., -1.0031025 ,
         -0.45583406, -1.647509  ],
        ...,
        [-0.10952955, -0.57149386, -1.4142135 , ..., -0.5008921 ,
         -0.45583406, -1.647509  ],
        [ 0.77810806, -0.37315318, -1.4142135 , ..., -1.0031025 ,
         -0.45583406, -1.647509  ],
        [ 0.9257272 , -0.39440045, -1.4142135 , ..., -1.0031025 ,
         -0.45583406, -1.647509  ]],

       [[-0.7479397 ,  1.8678857 , -1.4142135 , ..., -1.0031025 ,
         -0.45583406, -1.474087  ],
        [-1.2349257 ,  2.1685455 , -1.4142135 , ..., -1.0031025 ,
         -0.45583406, -1.474087  ],
        [-0.9121866 ,  2.0892835 , -1.4142135 , ..., -1.0031025 ,
         -0.45583406, -1.474087  ],
        ..